In [1]:
import gurobipy as gp
from gurobipy import GRB
try:
    model = gp.Model("test")
    print("Gurobi license is active and working!")
except Exception as e:
    print(f"Error: {e}")

Set parameter Username
Set parameter LicenseID to value 2691038
Academic license - for non-commercial use only - expires 2026-07-27
Gurobi license is active and working!


In [2]:
# Cell 1: imports, paths, load CAB25 costs and demands, quick checks

import numpy as np
import pandas as pd
import re
from pathlib import Path

# Your local paths
CAB_DIR = Path("/Users/aryaaghakoochek/Downloads/CAB")
OKELLY_FILE = CAB_DIR / "okelly.txt"
CAB25_FILE = CAB_DIR / "CAB25.txt"

print("Files exist:",
      "okelly" , OKELLY_FILE.exists(),
      "cab25", CAB25_FILE.exists())

def load_cab25(path: Path):
    txt = path.read_text()
    lines = [ln.strip() for ln in txt.splitlines() if ln.strip() != ""]
    n = int(lines[0])
    vals = []
    for ln in lines[1:]:
        for tok in re.split(r"[\t\s]+", ln):
            if tok != "":
                try:
                    vals.append(float(tok))
                except ValueError:
                    pass
    arr = np.array(vals).reshape(2, n, n)
    c = arr[0]  # transportation cost per unit
    w = arr[1]  # demand
    return n, c, w

N, C, W_raw = load_cab25(CAB25_FILE)
print("N:", N)
print("C shape:", C.shape, "W shape:", W_raw.shape)
print("Cost stats: min", C.min(), "max", C.max(), "mean", C.mean())
print("Demand stats: min", W_raw.min(), "max", W_raw.max(), "sum", W_raw.sum(), "mean", W_raw.mean())

# Normalize demands to sum to 1. The paper reports satisfied demand in percent, so this helps.
W = W_raw / W_raw.sum()
print("W normalized sum:", W.sum())

# Basic origin and destination totals
Oi = W.sum(axis=1)
Dj = W.sum(axis=0)
print("Top 5 origin nodes by share:", np.argsort(-Oi)[:5], Oi[np.argsort(-Oi)[:5]])
print("Top 5 destination nodes by share:", np.argsort(-Dj)[:5], Dj[np.argsort(-Dj)[:5]])


Files exist: okelly True cab25 True
N: 25
C shape: (25, 25) W shape: (25, 25)
Cost stats: min 0.0 max 205088.0 mean 13664.0096
Demand stats: min 0.0 max 2725.790039 sum 640873.9474160001 mean 1025.3983158656001
W normalized sum: 0.9999999999999999
Top 5 origin nodes by share: [22 21 11 18 13] [0.07051998 0.06958812 0.06426333 0.05451678 0.04709011]
Top 5 destination nodes by share: [22 21 11 18 13] [0.07051998 0.06958812 0.06426333 0.05451678 0.04709011]


In [3]:
# Cell 2: parameter factory for CAB experiments

def cab_params(revenue_level: str, cost_level: str, alpha: float):
    """
    Create parameter arrays per Taherkhani & Alumur for CAB:
      revenues r in {1000,1500,2000}
      hub fixed cost f in {50,100,150}
      inter-hub fixed g = 0.1 f
      direct fixed q = 0.2 g
      alpha in {0.2,0.4,0.6,0.8}
    Returns R, F, G, Q, alpha
    """
    rev_map = {"low": 1000.0, "medium": 1500.0, "high": 2000.0}
    f_map = {"low": 50.0, "medium": 100.0, "high": 150.0}
    if revenue_level not in rev_map:
        raise ValueError("revenue_level must be one of low, medium, high")
    if cost_level not in f_map:
        raise ValueError("cost_level must be one of low, medium, high")
    r = rev_map[revenue_level]
    f = f_map[cost_level]
    g = 0.1 * f
    q = 0.2 * g
    R = np.full((N, N), r, dtype=float)
    F = np.full(N, f, dtype=float)
    G = np.full((N, N), g, dtype=float)
    Q = np.full((N, N), q, dtype=float)
    return R, F, G, Q, float(alpha)

# Quick preview for the benchmark case: high revenue, low cost, alpha=0.2
R, F, G, Q, alpha = cab_params("high", "low", 0.2)
print("R unique:", np.unique(R)[:3], "F unique:", np.unique(F), "G unique:", np.unique(G), "Q unique:", np.unique(Q), "alpha:", alpha)


R unique: [2000.] F unique: [50.] G unique: [5.] Q unique: [1.] alpha: 0.2


In [8]:
# Cell 3d: diagnose profitability of hub paths with given C and alpha

def best_path_cost_stats(Rval=2000.0, alpha=0.2):
    N = C.shape[0]
    best_cost = np.full((N, N), np.inf)
    best_tuple = [[None]*N for _ in range(N)]
    for i in range(N):
        for j in range(N):
            if i == j:
                best_cost[i, j] = 0.0
                best_tuple[i][j] = (None, None)
                continue
            # search over all k,l
            bc = np.inf
            bt = None
            for k in range(N):
                cik = C[i, k]
                for l in range(N):
                    cost = cik + alpha * C[k, l] + C[l, j]
                    if cost < bc:
                        bc = cost
                        bt = (k, l)
            best_cost[i, j] = bc
            best_tuple[i][j] = bt
    # counts where profitable
    prof_mask = best_cost < Rval
    num_prof = int(np.sum(prof_mask) - np.sum(np.eye(N)))  # exclude i=j
    total_pairs = N*N - N
    avg_best_cost = np.mean(best_cost[~np.isinf(best_cost)])
    return {
        "R": Rval,
        "alpha": alpha,
        "num_profitable_pairs": num_prof,
        "total_pairs": total_pairs,
        "pct_profitable": 100.0 * num_prof / total_pairs,
        "avg_best_cost": float(avg_best_cost),
        "min_best_cost": float(np.min(best_cost + np.where(best_cost==np.inf, 1e18, 0))),
        "max_best_cost": float(np.max(np.where(best_cost==np.inf, 0, best_cost)))
    }, best_cost, best_tuple

diag, best_cost, best_tuple = best_path_cost_stats(Rval=2000.0, alpha=0.2)
print(diag)

# Also compute expected net profit if we routed every OD by its cheapest hub path
est_profit = 2000.0 * (W.sum() - np.sum(W*np.eye(N))) - np.sum(W * best_cost)
print("Naive profit if all OD served via cheapest hub path:", est_profit)

# Show a few cheapest profitable pairs
pairs = []
for i in range(N):
    for j in range(N):
        if i==j: continue
        if best_cost[i,j] < 2000.0:
            k,l = best_tuple[i][j]
            pairs.append((i,j,k,l,best_cost[i,j], 2000.0 - best_cost[i,j]))
pairs_sorted = sorted(pairs, key=lambda x: x[5], reverse=True)[:10]
pairs_sorted[:10]


{'R': 2000.0, 'alpha': 0.2, 'num_profitable_pairs': 438, 'total_pairs': 600, 'pct_profitable': 73.0, 'avg_best_cost': 1628.27648, 'min_best_cost': 0.0, 'max_best_cost': 13249.0}
Naive profit if all OD served via cheapest hub path: 363.55381262844116


[(1, 24, 1, 24, 113.0, 1887.0),
 (24, 1, 24, 1, 113.0, 1887.0),
 (18, 23, 18, 23, 113.80000000000001, 1886.2),
 (23, 18, 23, 18, 113.80000000000001, 1886.2),
 (12, 18, 12, 18, 151.8, 1848.2),
 (18, 12, 18, 12, 151.8, 1848.2),
 (1, 18, 1, 18, 161.20000000000002, 1838.8),
 (18, 1, 18, 1, 161.20000000000002, 1838.8),
 (15, 18, 15, 18, 177.20000000000002, 1822.8),
 (18, 15, 18, 15, 177.20000000000002, 1822.8)]

In [9]:
# Cell 4: scale C so average best hub-path cost ≈ 810, then solve full multiple allocation

from time import time

C_base = C.copy()

def compute_best_cost_stats(Cmat, alpha, Rval=2000.0):
    N = Cmat.shape[0]
    best_cost = np.full((N, N), np.inf)
    for i in range(N):
        for j in range(N):
            if i == j:
                best_cost[i, j] = 0.0
                continue
            bc = np.inf
            for k in range(N):
                cik = Cmat[i, k]
                for l in range(N):
                    cost = cik + alpha * Cmat[k, l] + Cmat[l, j]
                    if cost < bc:
                        bc = cost
            best_cost[i, j] = bc
    mask = ~np.isinf(best_cost)
    return float(best_cost[mask].mean())

target_avg_cost = 810.0  # aimed so that 2000 - 810 ≈ 1190 net
cur_avg_cost = compute_best_cost_stats(C_base, alpha=0.2, Rval=2000.0)
tau = target_avg_cost / cur_avg_cost
print(f"Current avg best cost: {cur_avg_cost:.4f}, target: {target_avg_cost:.1f}, tau: {tau:.6f}")

C_scaled = C_base * tau
scaled_avg = compute_best_cost_stats(C_scaled, alpha=0.2)
print(f"Scaled avg best cost at alpha=0.2: {scaled_avg:.4f}")

# Rebuild and solve the full multiple model using C_scaled
def build_multiple_no_direct_withC(R, F, G, alpha, Cmat, W):
    N = Cmat.shape[0]
    nodes = range(N)
    m = gp.Model("PMHLP_mult_nodir_full_scaled")

    h = m.addVars(nodes, vtype=GRB.BINARY, name="h")
    z = m.addVars(nodes, nodes, vtype=GRB.BINARY, name="z")

    cand = []
    for i in nodes:
        for j in nodes:
            if i == j:
                continue
            for k in nodes:
                cik = Cmat[i, k]
                for l in nodes:
                    if R[i, j] >= (cik + Cmat[l, j]):
                        cand.append((i, j, k, l))
    print(f"|y candidates| (scaled) = {len(cand)}")
    y = m.addVars(cand, vtype=GRB.BINARY, name="y")

    f = m.addVars(nodes, nodes, nodes, vtype=GRB.CONTINUOUS, lb=0.0, name="f")

    rev = gp.quicksum(R[i, j] * W[i, j] * y[i, j, k, l] for (i, j, k, l) in cand)
    cost_cd = gp.quicksum((Cmat[i, k] + Cmat[l, j]) * W[i, j] * y[i, j, k, l] for (i, j, k, l) in cand)
    cost_trans = alpha * gp.quicksum(Cmat[k, l] * f[i, k, l] for i in nodes for k in nodes for l in nodes)
    fix_hub = gp.quicksum(F[k] * h[k] for k in nodes)
    fix_link = gp.quicksum(G[k, l] * z[k, l] for k in nodes for l in nodes if k != l)

    m.setObjective(rev - cost_cd - cost_trans - fix_hub - fix_link, GRB.MAXIMIZE)

    for i in nodes:
        for j in nodes:
            if i == j:
                continue
            m.addConstr(gp.quicksum(y[i, j, k, l]
                                    for k in nodes for l in nodes if (i, j, k, l) in y) <= 1.0)

    for (i, j, k, l) in cand:
        m.addConstr(y[i, j, k, l] <= h[k])
        m.addConstr(y[i, j, k, l] <= h[l])
        m.addConstr(y[i, j, k, l] <= z[k, l])

    Wtot = float(W.sum())
    for i in nodes:
        for k in nodes:
            outflow = gp.quicksum(f[i, k, l] for l in nodes if l != k)
            inflow  = gp.quicksum(f[i, l, k] for l in nodes if l != k)
            used_out = gp.quicksum(W[i, j] * y[i, j, k, l] for j in nodes for l in nodes if (i, j, k, l) in y)
            used_in  = gp.quicksum(W[i, j] * y[i, j, l, k] for j in nodes for l in nodes if (i, j, l, k) in y)
            m.addConstr(inflow + used_out == outflow + used_in)

    for i in nodes:
        for k in nodes:
            for l in nodes:
                if k == l:
                    continue
                m.addConstr(f[i, k, l] <= Wtot * z[k, l])

    return m, h, z, y, f, cand

def solve_mult_nodir_full_withC(Cmat, revenue_level, cost_level, alpha,
                                timelimit=3600, mipgap=1e-5, threads=None):
    R, F, G, Q, alpha = cab_params(revenue_level, cost_level, alpha)
    m, h, z, y, f, cand = build_multiple_no_direct_withC(R, F, G, alpha, Cmat, W)
    m.Params.TimeLimit = timelimit
    m.Params.MIPGap = mipgap
    if threads is not None:
        m.Params.Threads = threads

    t0 = time()
    m.optimize()
    dt = time() - t0

    obj = None
    sat = 0.0
    if m.SolCount > 0:
        obj = m.ObjVal
        for (i, j, k, l), var in y.items():
            if var.X > 0.5:
                sat += W[i, j]
    sat_pct = 100.0 * sat / W.sum()

    return {
        "obj": obj,
        "sat_pct": sat_pct,
        "time_sec": dt,
        "gap": m.MIPGap if m.SolCount > 0 else None,
        "vars_y": len(cand),
        "nodes_explored": m.NodeCount,
        "tau": tau,
        "avg_best_cost_scaled": scaled_avg
    }

res_full_scaled = solve_mult_nodir_full_withC(C_scaled, "high", "low", 0.2, timelimit=3600)
print(res_full_scaled)


Current avg best cost: 1628.2765, target: 810.0, tau: 0.497459
Scaled avg best cost at alpha=0.2: 810.0000
|y candidates| (scaled) = 15722
Set parameter TimeLimit to value 3600
Set parameter MIPGap to value 1e-05
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (mac64[x86] - Darwin 22.6.0 22H527)

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Non-default parameters:
TimeLimit  3600
MIPGap  1e-05

Optimize a model with 63391 rows, 31997 columns and 199862 nonzeros
Model fingerprint: 0x54919030
Variable types: 15625 continuous, 16372 integer (16372 binary)
Coefficient statistics:
  Matrix range     [6e-05, 1e+00]
  Objective range  [9e-04, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 3653 rows and 650 columns
Presolve time: 2.82s
Presolved: 59738 rows, 31347 columns, 193617 nonzeros
Variable types: 15000 continu

In [10]:
# Cell 5: scale fixed costs by tau as well, resolve, and report objective components

from time import time
import gurobipy as gp
from gurobipy import GRB

def solve_mult_nodir_full_withC_and_scaled_fixed(Cmat, revenue_level, cost_level, alpha, tau,
                                                 timelimit=3600, mipgap=1e-5, threads=None):
    # Base params
    R, F_base, G_base, Q_base, alpha = cab_params(revenue_level, cost_level, alpha)
    # Scale fixed costs by tau to preserve proportions after transport scaling
    F = F_base * tau
    G = G_base * tau

    N = Cmat.shape[0]
    nodes = range(N)
    m = gp.Model("PMHLP_mult_nodir_full_scaled_fixed")

    h = m.addVars(nodes, vtype=GRB.BINARY, name="h")
    z = m.addVars(nodes, nodes, vtype=GRB.BINARY, name="z")

    cand = []
    for i in nodes:
        for j in nodes:
            if i == j:
                continue
            for k in nodes:
                cik = Cmat[i, k]
                for l in nodes:
                    if R[i, j] >= (cik + Cmat[l, j]):
                        cand.append((i, j, k, l))
    print(f"|y candidates| (scaled C, scaled fixed) = {len(cand)}")
    y = m.addVars(cand, vtype=GRB.BINARY, name="y")

    f = m.addVars(nodes, nodes, nodes, vtype=GRB.CONTINUOUS, lb=0.0, name="f")

    # Objective terms we will track
    rev_expr       = gp.quicksum(R[i, j] * W[i, j] * y[i, j, k, l] for (i, j, k, l) in cand)
    cost_cd_expr   = gp.quicksum((Cmat[i, k] + Cmat[l, j]) * W[i, j] * y[i, j, k, l] for (i, j, k, l) in cand)
    cost_tr_expr   = alpha * gp.quicksum(Cmat[k, l] * f[i, k, l] for i in nodes for k in nodes for l in nodes)
    fix_hub_expr   = gp.quicksum(F[k] * h[k] for k in nodes)
    fix_link_expr  = gp.quicksum(G[k, l] * z[k, l] for k in nodes for l in nodes if k != l)

    m.setObjective(rev_expr - cost_cd_expr - cost_tr_expr - fix_hub_expr - fix_link_expr, GRB.MAXIMIZE)

    for i in nodes:
        for j in nodes:
            if i == j:
                continue
            m.addConstr(gp.quicksum(y[i, j, k, l]
                                    for k in nodes for l in nodes if (i, j, k, l) in y) <= 1.0)

    for (i, j, k, l) in cand:
        m.addConstr(y[i, j, k, l] <= h[k])
        m.addConstr(y[i, j, k, l] <= h[l])
        m.addConstr(y[i, j, k, l] <= z[k, l])

    Wtot = float(W.sum())
    for i in nodes:
        for k in nodes:
            outflow = gp.quicksum(f[i, k, l] for l in nodes if l != k)
            inflow  = gp.quicksum(f[i, l, k] for l in nodes if l != k)
            used_out = gp.quicksum(W[i, j] * y[i, j, k, l] for j in nodes for l in nodes if (i, j, k, l) in y)
            used_in  = gp.quicksum(W[i, j] * y[i, j, l, k] for j in nodes for l in nodes if (i, j, l, k) in y)
            m.addConstr(inflow + used_out == outflow + used_in)

    for i in nodes:
        for k in nodes:
            for l in nodes:
                if k == l:
                    continue
                m.addConstr(f[i, k, l] <= Wtot * z[k, l])

    # Params
    m.Params.TimeLimit = timelimit
    m.Params.MIPGap = mipgap
    if threads is not None:
        m.Params.Threads = threads

    t0 = time()
    m.optimize()
    dt = time() - t0

    obj = None
    sat = 0.0
    rev = cd = tr = fh = fl = None
    open_hubs = [k for k in nodes if h[k].X > 0.5] if m.SolCount > 0 else []
    if m.SolCount > 0:
        obj = m.ObjVal
        rev = rev_expr.getValue()
        cd  = cost_cd_expr.getValue()
        tr  = cost_tr_expr.getValue()
        fh  = fix_hub_expr.getValue()
        fl  = fix_link_expr.getValue()
        for (i, j, k, l), var in y.items():
            if var.X > 0.5:
                sat += W[i, j]
    sat_pct = 100.0 * sat / W.sum()

    return {
        "obj": obj,
        "sat_pct": sat_pct,
        "time_sec": dt,
        "gap": m.MIPGap if m.SolCount > 0 else None,
        "open_hubs": open_hubs,
        "rev": rev,
        "cost_cd": cd,
        "cost_trans": tr,
        "fix_hub": fh,
        "fix_link": fl,
        "tau": tau
    }

res_scaled_fixed = solve_mult_nodir_full_withC_and_scaled_fixed(
    C_scaled, "high", "low", 0.2, tau, timelimit=3600
)
print(res_scaled_fixed)


|y candidates| (scaled C, scaled fixed) = 15722
Set parameter TimeLimit to value 3600
Set parameter MIPGap to value 1e-05
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (mac64[x86] - Darwin 22.6.0 22H527)

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Non-default parameters:
TimeLimit  3600
MIPGap  1e-05

Optimize a model with 63391 rows, 31997 columns and 199862 nonzeros
Model fingerprint: 0x3a3c6535
Variable types: 15625 continuous, 16372 integer (16372 binary)
Coefficient statistics:
  Matrix range     [6e-05, 1e+00]
  Objective range  [9e-04, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 3653 rows and 650 columns
Presolve time: 2.89s
Presolved: 59738 rows, 31347 columns, 193617 nonzeros
Variable types: 15000 continuous, 16347 integer (16347 binary)

Deterministic concurrent LP optimizer: primal and dual s

In [12]:
# Cell 6: warm start heuristic + gamma scaling for fixed costs (fixed syntax)

import numpy as np
import gurobipy as gp
from gurobipy import GRB
from time import time

def warm_start_construct(R, F, G, alpha, Cmat, W, allow_links=True):
    N = Cmat.shape[0]
    nodes = range(N)
    Hset = set()
    Zset = set()
    Yset = set()

    Oi = W.sum(axis=1)
    Dj = W.sum(axis=0)
    score = Oi + Dj
    order = list(np.argsort(-score))

    for k in order[:4]:
        Hset.add(k)

    for i in nodes:
        for j in nodes:
            if i == j or W[i, j] == 0:
                continue
            best_val = -1e18
            best_pair = None
            for k in Hset:
                cik = Cmat[i, k]
                for l in Hset:
                    val = R[i, j] - (cik + alpha * Cmat[k, l] + Cmat[l, j])
                    if val > best_val:
                        best_val = val
                        best_pair = (k, l)
            if best_val <= 0:
                bc = 1e18
                bp = None
                for k in nodes:
                    cik = Cmat[i, k]
                    for l in nodes:
                        cost = cik + alpha * Cmat[k, l] + Cmat[l, j]
                        if cost < bc:
                            bc = cost
                            bp = (k, l)
                if bp is not None and R[i, j] - bc > 0:
                    k, l = bp
                    Hset.add(k); Hset.add(l)
                    best_pair = bp
                    best_val = R[i, j] - bc

            if best_pair and best_val > 0:
                k, l = best_pair
                Yset.add((i, j, k, l))
                if allow_links:
                    Zset.add((k, l))
                Hset.add(k); Hset.add(l)

    return Hset, Zset, Yset

def apply_mip_start(h, z, y, start_sets):
    Hset, Zset, Yset = start_sets
    N = len(h)
    for k in range(N):
        h[k].start = 1.0 if k in Hset else 0.0
    for (k, l), var in z.items():
        if k == l:
            var.start = 0.0
        else:
            var.start = 1.0 if (k, l) in Zset else 0.0
    for key, var in y.items():
        i, j, k, l = key
        var.start = 1.0 if (i, j, k, l) in Yset else 0.0

def solve_mult_nodir_full_withC_gamma(Cmat, revenue_level, cost_level, alpha, tau, gamma=1.0,
                                      timelimit=3600, mipgap=1e-5, warm=True, threads=None):
    R, F_base, G_base, Q_base, alpha = cab_params(revenue_level, cost_level, alpha)
    F = F_base * tau * gamma
    G = G_base * tau * gamma

    N = Cmat.shape[0]
    nodes = range(N)
    m = gp.Model("PMHLP_mult_nodir_full_scaled_gamma")

    h = m.addVars(nodes, vtype=GRB.BINARY, name="h")
    z = m.addVars(nodes, nodes, vtype=GRB.BINARY, name="z")

    cand = []
    for i in nodes:
        for j in nodes:
            if i == j:
                continue
            for k in nodes:
                cik = Cmat[i, k]
                for l in nodes:
                    if R[i, j] >= (cik + Cmat[l, j]):
                        cand.append((i, j, k, l))
    y = m.addVars(cand, vtype=GRB.BINARY, name="y")
    f = m.addVars(nodes, nodes, nodes, vtype=GRB.CONTINUOUS, lb=0.0, name="f")

    rev_expr     = gp.quicksum(R[i, j] * W[i, j] * y[i, j, k, l] for (i, j, k, l) in cand)
    cost_cd_expr = gp.quicksum((Cmat[i, k] + Cmat[l, j]) * W[i, j] * y[i, j, k, l] for (i, j, k, l) in cand)
    cost_tr_expr = alpha * gp.quicksum(Cmat[k, l] * f[i, k, l] for i in nodes for k in nodes for l in nodes)
    fix_hub_expr = gp.quicksum(F[k] * h[k] for k in nodes)
    fix_link_expr= gp.quicksum(G[k, l] * z[k, l] for k in nodes for l in nodes if k != l)

    m.setObjective(rev_expr - cost_cd_expr - cost_tr_expr - fix_hub_expr - fix_link_expr, GRB.MAXIMIZE)

    for i in nodes:
        for j in nodes:
            if i == j:
                continue
            m.addConstr(gp.quicksum(y[i, j, k, l] for k in nodes for l in nodes if (i, j, k, l) in y) <= 1.0)

    for (i, j, k, l) in cand:
        m.addConstr(y[i, j, k, l] <= h[k])
        m.addConstr(y[i, j, k, l] <= h[l])
        m.addConstr(y[i, j, k, l] <= z[k, l])

    Wtot = float(W.sum())
    for i in nodes:
        for k in nodes:
            outflow = gp.quicksum(f[i, k, l] for l in nodes if l != k)
            inflow  = gp.quicksum(f[i, l, k] for l in nodes if l != k)
            used_out = gp.quicksum(W[i, j] * y[i, j, k, l] for j in nodes for l in nodes if (i, j, k, l) in y)
            used_in  = gp.quicksum(W[i, j] * y[i, j, l, k] for j in nodes for l in nodes if (i, j, l, k) in y)
            m.addConstr(inflow + used_out == outflow + used_in)

    for i in nodes:
        for k in nodes:
            for l in nodes:
                if k == l:
                    continue
                m.addConstr(f[i, k, l] <= Wtot * z[k, l])

    m.Params.TimeLimit = timelimit
    m.Params.MIPGap = mipgap
    if threads is not None:
        m.Params.Threads = threads

    if warm:
        Hset, Zset, Yset = warm_start_construct(R, F, G, alpha, Cmat, W)
        apply_mip_start(h, z, y, (Hset, Zset, Yset))

    t0 = time()
    m.optimize()
    dt = time() - t0

    obj = None
    sat = 0.0
    rev = cd = tr = fh = fl = None
    open_hubs = [k for k in nodes if h[k].X > 0.5] if m.SolCount > 0 else []
    if m.SolCount > 0:
        obj = m.ObjVal
        rev = rev_expr.getValue()
        cd  = cost_cd_expr.getValue()
        tr  = cost_tr_expr.getValue()
        fh  = fix_hub_expr.getValue()
        fl  = fix_link_expr.getValue()
        for (i, j, k, l), var in y.items():
            if var.X > 0.5:
                sat += W[i, j]
    sat_pct = 100.0 * sat / W.sum()

    return {
        "obj": obj,
        "sat_pct": sat_pct,
        "time_sec": dt,
        "gap": m.MIPGap if m.SolCount > 0 else None,
        "open_hubs": open_hubs,
        "rev": rev,
        "cost_cd": cd,
        "cost_trans": tr,
        "fix_hub": fh,
        "fix_link": fl,
        "gamma": gamma
    }

# Try gamma = 0.6 with warm start
res_gamma06_warm = solve_mult_nodir_full_withC_gamma(
    C_scaled, "high", "low", 0.2, tau, gamma=0.6, timelimit=3600, warm=True
)
print(res_gamma06_warm)


Set parameter TimeLimit to value 3600
Set parameter MIPGap to value 1e-05
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (mac64[x86] - Darwin 22.6.0 22H527)

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Non-default parameters:
TimeLimit  3600
MIPGap  1e-05

Optimize a model with 63391 rows, 31997 columns and 199862 nonzeros
Model fingerprint: 0x2625ccae
Variable types: 15625 continuous, 16372 integer (16372 binary)
Coefficient statistics:
  Matrix range     [6e-05, 1e+00]
  Objective range  [9e-04, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

User MIP start did not produce a new incumbent solution
User MIP start violates constraint R674 by 1.000000000

Found heuristic solution: objective -0.0000000
Presolve removed 3653 rows and 650 columns
Presolve time: 3.24s
Presolved: 59738 rows, 31347 columns, 193617 nonzeros
Variable types: 15000 continuous, 16347 integer (16347 

  1899   464  697.37444    6  412  692.60415  697.96849  0.77%  90.0  145s
  2121   531  694.92092   11  766  692.60415  697.80386  0.75%  87.6  152s
  2243   542     cutoff   14       692.60415  697.77084  0.75%  86.1  155s
  2484   548  694.44059   12  255  692.60415  697.32749  0.68%  84.0  161s
H 2496   544                     692.7008407  697.32749  0.67%  84.0  161s
  2636   586  693.98665   17  225  692.70084  697.28295  0.66%  82.9  167s
  2800   601  693.25181   16 1307  692.70084  697.18083  0.65%  81.3  170s
  3172   665     cutoff   16       692.70084  696.75489  0.59%  77.5  177s
  3341   720  693.10522   17 1411  692.70084  696.68875  0.58%  76.4  180s
  3536   721  695.16290   11 1854  692.70084  696.65457  0.57%  75.0  220s
H 3537   685                     692.7114590  696.65457  0.57%  75.0  252s
H 3537   651                     692.7496783  696.65457  0.56%  75.0  252s
  3539   652  695.59777   10 1226  692.74968  696.65457  0.56%  74.9  303s
H 3539   619             

In [13]:
# Cell 7: sweep gamma to push satisfied demand toward ~100%

gammas = [0.55, 0.50, 0.45, 0.40, 0.35]
results = []

for g in gammas:
    print("\n=== Running gamma =", g, "===")
    res = solve_mult_nodir_full_withC_gamma(
        C_scaled, "high", "low", 0.2, tau,
        gamma=g, timelimit=300, mipgap=1e-4, warm=True
    )
    results.append(res)
    print(res)

df_gamma = pd.DataFrame(results)
df_gamma



=== Running gamma = 0.55 ===
Set parameter TimeLimit to value 300
Set parameter MIPGap to value 0.0001
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (mac64[x86] - Darwin 22.6.0 22H527)

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Non-default parameters:
TimeLimit  300

Optimize a model with 63391 rows, 31997 columns and 199862 nonzeros
Model fingerprint: 0xd1174509
Variable types: 15625 continuous, 16372 integer (16372 binary)
Coefficient statistics:
  Matrix range     [6e-05, 1e+00]
  Objective range  [9e-04, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

User MIP start did not produce a new incumbent solution
User MIP start violates constraint R674 by 1.000000000

Found heuristic solution: objective -0.0000000
Presolve removed 3653 rows and 650 columns
Presolve time: 2.99s
Presolved: 59738 rows, 31347 columns, 193617 nonzeros
Variable types: 15000 continuous, 16347 

  2286   487     cutoff   11       734.30685  737.99489  0.50%  51.6  121s
  2501   518  736.60410   12  260  734.30685  737.81670  0.48%  52.2  126s
  2752   590  736.14594   12  244  734.30685  737.64818  0.46%  52.1  130s
  2975   620  736.38635   14  792  734.30685  737.54729  0.44%  52.6  135s
  3217   654     cutoff   18       734.30685  737.40571  0.42%  53.1  141s
  3469   665  735.62865   13  843  734.30685  737.30747  0.41%  53.7  147s
  3603   660  734.64969   17  699  734.30685  737.28777  0.41%  54.1  151s
  3741   675     cutoff   18       734.30685  737.16443  0.39%  55.1  155s
  4048   676  734.55251   13  918  734.30685  736.98604  0.36%  56.2  162s
H 4085   662                     734.3823332  736.98604  0.35%  56.0  162s
H 4134   662                     734.3823818  736.98604  0.35%  56.2  162s
  4146   661  734.53469   14  141  734.38238  736.90539  0.34%  56.3  166s
  4291   646     cutoff   12       734.38238  736.80452  0.33%  56.7  170s
  4644   657  734.66857  

H    0     0                     761.2154340  786.33178  3.30%     -   31s
     0     0  786.33178    0 1728  761.21543  786.33178  3.30%     -   31s
H    0     0                     761.2184278  786.32971  3.30%     -   31s
H    0     0                     762.2786198  786.32971  3.16%     -   31s
     0     0  786.32971    0 1688  762.27862  786.32971  3.16%     -   31s
H    0     0                     764.7990280  786.32971  2.82%     -   36s
H    0     0                     765.0363463  786.32971  2.78%     -   36s
H    0     0                     765.5042749  786.32971  2.72%     -   36s
H    0     0                     765.5042986  786.32971  2.72%     -   36s
H    0     0                     766.4936233  786.32971  2.59%     -   36s
H    0     0                     767.1296607  786.32971  2.50%     -   36s
H    0     0                     767.7180211  786.32971  2.42%     -   36s
H    0     0                     769.5977048  786.32971  2.17%     -   36s
     0     0  784.60829  

H    0     0                     785.6380783  830.90097  5.76%     -   22s
H    0     0                     790.0183045  830.90097  5.17%     -   22s
H    0     0                     790.0219564  830.90097  5.17%     -   22s
H    0     0                     791.2883081  830.90097  5.01%     -   22s
     0     0  830.90097    0 1700  791.28831  830.90097  5.01%     -   22s
     0     0  830.89208    0 1671  791.28831  830.89208  5.00%     -   23s
     0     0  830.42475    0 1691  791.28831  830.42475  4.95%     -   26s
H    0     0                     809.3043831  830.40212  2.61%     -   27s
H    0     0                     809.6702323  830.40212  2.56%     -   27s
H    0     0                     809.9321712  830.40212  2.53%     -   27s
H    0     0                     810.2352109  830.40212  2.49%     -   27s
H    0     0                     810.4606488  830.40212  2.46%     -   27s
     0     0  830.40212    0 1662  810.46065  830.40212  2.46%     -   27s
     0     0  830.40212  


    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  891.15880    0 1809   -0.00000  891.15880      -     -    6s
H    0     0                     588.2161491  891.15880  51.5%     -    7s
H    0     0                     668.2198334  891.15880  33.4%     -    7s
H    0     0                     670.5420123  891.15880  32.9%     -    7s
H    0     0                     675.8719960  891.15880  31.9%     -    7s
     0     0  880.19475    0  943  675.87200  880.19475  30.2%     -   12s
H    0     0                     729.3578324  880.17133  20.7%     -   12s
     0     0  880.06438    0  956  729.35783  880.06438  20.7%     -   13s
H    0     0                     729.6269558  880.06438  20.6%     -   14s
     0     0  878.33078    0  930  729.62696  878.33078  20.4%     -   15s
H    0     0                     776.2635879  878.15340  13.1%     -   16s
     0     0  878.15340

Showing primal log only...

Root relaxation presolved: 59738 rows, 31347 columns, 193617 nonzeros


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   10588    4.0460981e+02   0.000000e+00   1.158385e+07      5s
Concurrent spin time: 0.25s

Solved with primal simplex

Root relaxation: objective 9.363583e+02, 10711 iterations, 2.78 seconds (1.65 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  936.35828    0 1606   -0.00000  936.35828      -     -    6s
H    0     0                     676.0163039  936.35828  38.5%     -    6s
H    0     0                     745.9541853  936.35828  25.5%     -    6s
H    0     0                     747.8747821  936.35828  25.2%     -    6s
H    0     0                     752.7043786  936.35828  24.4%     -    6s
H    0     0                     778.2416466  930.93680  19.6%     -   10s
H 

,obj,sat_pct,time_sec,gap,open_hubs,rev,cost_cd,cost_trans,fix_hub,fix_link,gamma
0,734.382395,92.455484,255.522891,0.000000e+00,"[1, 2, 5, 6, 7, 8, 9, 10, 11, 12, 16, 17, 18, ...",1849.109679,317.133912,335.205682,232.561856,229.825834,0.55
1,776.417637,92.455484,119.812567,2.418556e-09,"[1, 2, 5, 6, 7, 8, 9, 10, 11, 12, 16, 17, 18, ...",1849.109678,317.133938,335.205657,211.419869,208.932576,0.50
2,821.372729,93.223056,205.706719,0.000000e+00,"[1, 2, 5, 6, 7, 8, 9, 10, 11, 12, 13, 16, 17, ...",1864.461128,243.477342,358.614084,212.663515,228.333458,0.45
3,870.620702,93.223056,122.841252,0.000000e+00,"[1, 2, 5, 6, 7, 8, 9, 10, 11, 12, 13, 16, 17, ...",1864.461128,237.098736,358.774877,189.034236,208.932576,0.40
4,923.488774,93.363570,87.485917,9.131682e-05,"[0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12, 13, 16, 1...",1867.271406,198.349416,366.742921,174.110480,204.579814,0.35


In [14]:
# Cell 8: run gamma = 0.30 to push demand served toward ~100%

res_gamma03_warm = solve_mult_nodir_full_withC_gamma(
    C_scaled, "high", "low", 0.2, tau, gamma=0.30, timelimit=300, mipgap=1e-4, warm=True
)
print(res_gamma03_warm)


Set parameter TimeLimit to value 300
Set parameter MIPGap to value 0.0001
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (mac64[x86] - Darwin 22.6.0 22H527)

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Non-default parameters:
TimeLimit  300

Optimize a model with 63391 rows, 31997 columns and 199862 nonzeros
Model fingerprint: 0xde40a0e2
Variable types: 15625 continuous, 16372 integer (16372 binary)
Coefficient statistics:
  Matrix range     [6e-05, 1e+00]
  Objective range  [9e-04, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

User MIP start did not produce a new incumbent solution
User MIP start violates constraint R674 by 1.000000000

Found heuristic solution: objective -0.0000000
Presolve removed 3653 rows and 650 columns
Presolve time: 2.66s
Presolved: 59738 rows, 31347 columns, 193617 nonzeros
Variable types: 15000 continuous, 16347 integer (16347 binary)

Determ

In [15]:
# Cell 9: Multiple-allocation WITH direct links, warm start, and small parameter sweep

import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd
from time import time

def construct_q_matrix(G_base, tau, q_scale):
    """Make a simple direct-link fixed-cost matrix q_ij = mean(G_base) * tau * q_scale."""
    q0 = float(G_base.mean()) * float(tau) * float(q_scale)
    n = G_base.shape[0]
    q = np.full((n, n), q0, dtype=float)
    np.fill_diagonal(q, 0.0)
    return q

def best_hub_cost_matrix(Cmat, alpha):
    """best hub path cost for each (i,j): min_{k,l} [Cik + alpha Ckl + Clj]."""
    N = Cmat.shape[0]
    best = np.full((N, N), np.inf)
    for i in range(N):
        for j in range(N):
            if i == j:
                best[i, j] = 0.0
                continue
            bc = np.inf
            for k in range(N):
                cik = Cmat[i, k]
                for l in range(N):
                    c = cik + alpha * Cmat[k, l] + Cmat[l, j]
                    if c < bc:
                        bc = c
            best[i, j] = bc
    return best

def warm_start_direct(R, F, G, q, alpha, Cmat, W):
    """
    Heuristic:
    - Start with hubs = top-traffic nodes.
    - For each OD, compare best hub-path net vs direct net; pick the better if positive.
    """
    N = Cmat.shape[0]
    nodes = range(N)
    Oi = W.sum(axis=1)
    Dj = W.sum(axis=0)
    score = Oi + Dj
    order = list(np.argsort(-score))

    Hset = set(order[:4])
    Zset = set()
    Yset = set()
    Sset = set()

    besthub = best_hub_cost_matrix(Cmat, alpha)

    for i in nodes:
        for j in nodes:
            if i == j or W[i, j] <= 0:
                continue

            # Direct net profit if we opened s_ij alone (ignores hub/h costs interactions)
            net_dir = (R[i, j] - Cmat[i, j]) * W[i, j] - q[i, j]

            # Best hub net using current Hset (approximate; allow k,l anywhere if none in Hset works)
            bc_curr = np.inf
            best_pair = None
            for k in Hset:
                cik = Cmat[i, k]
                for l in Hset:
                    c = cik + alpha * Cmat[k, l] + Cmat[l, j]
                    if c < bc_curr:
                        bc_curr = c
                        best_pair = (k, l)
            # If no good in Hset, fall back to global best
            if not np.isfinite(bc_curr):
                bc_curr = besthub[i, j]
                # also take the k,l that achieves it
                # quick search:
                for k in nodes:
                    for l in nodes:
                        c = Cmat[i, k] + alpha * Cmat[k, l] + Cmat[l, j]
                        if abs(c - bc_curr) < 1e-9:
                            best_pair = (k, l); break
                    if best_pair: break

            net_hub = (R[i, j] - bc_curr) * W[i, j]

            if net_dir > max(0.0, net_hub):
                # prefer direct if profitable and better
                Sset.add((i, j))
            elif net_hub > 0:
                # use hubs
                Yset.add((i, j, best_pair[0], best_pair[1]))
                Hset.add(best_pair[0]); Hset.add(best_pair[1])
                Zset.add((best_pair[0], best_pair[1]))

    return Hset, Zset, Yset, Sset

def build_solve_multiple_direct(Cmat, R, F, G, q, alpha, W,
                                timelimit=180, mipgap=1e-4, warm=True, threads=None):
    N = Cmat.shape[0]
    nodes = range(N)

    m = gp.Model("PMHLP_mult_direct")

    h = m.addVars(nodes, vtype=GRB.BINARY, name="h")
    z = m.addVars(nodes, nodes, vtype=GRB.BINARY, name="z")
    s = m.addVars(nodes, nodes, vtype=GRB.BINARY, name="s")  # direct link
    # Candiate reduction: forbid hub path when revenue < Cik + Clj (collection+distribution)
    cand = []
    for i in nodes:
        for j in nodes:
            if i == j: 
                continue
            for k in nodes:
                cik = Cmat[i, k]
                for l in nodes:
                    if R[i, j] >= (cik + Cmat[l, j]):
                        cand.append((i, j, k, l))
    y = m.addVars(cand, vtype=GRB.BINARY, name="y")
    f = m.addVars(nodes, nodes, nodes, vtype=GRB.CONTINUOUS, lb=0.0, name="f")

    rev_hub = gp.quicksum(R[i, j] * W[i, j] * y[i, j, k, l] for (i, j, k, l) in cand)
    rev_dir = gp.quicksum(R[i, j] * W[i, j] * s[i, j] for i in nodes for j in nodes if i != j)

    cost_cd  = gp.quicksum((Cmat[i, k] + Cmat[l, j]) * W[i, j] * y[i, j, k, l] for (i, j, k, l) in cand)
    cost_tr  = alpha * gp.quicksum(Cmat[k, l] * f[i, k, l] for i in nodes for k in nodes for l in nodes if k != l)
    cost_dir = gp.quicksum(W[i, j] * Cmat[i, j] * s[i, j] for i in nodes for j in nodes if i != j)

    fix_hub  = gp.quicksum(F[k] * h[k] for k in nodes)
    fix_link = gp.quicksum(G[k, l] * z[k, l] for k in nodes for l in nodes if k != l)
    fix_dir  = gp.quicksum(q[i, j] * s[i, j] for i in nodes for j in nodes if i != j)

    m.setObjective((rev_hub + rev_dir) - (cost_cd + cost_tr + cost_dir + fix_hub + fix_link + fix_dir),
                   GRB.MAXIMIZE)

    # Each OD can be served at most once via hubs; direct is an alternative, so:
    for i in nodes:
        for j in nodes:
            if i == j: 
                continue
            m.addConstr(gp.quicksum(y[i, j, k, l] for k in nodes for l in nodes if (i, j, k, l) in y)
                        + s[i, j] <= 1.0)

    # y implies hubs and link
    for (i, j, k, l) in cand:
        m.addConstr(y[i, j, k, l] <= h[k])
        m.addConstr(y[i, j, k, l] <= h[l])
        m.addConstr(y[i, j, k, l] <= z[k, l])

    # direct only between non-hubs
    for i in nodes:
        for j in nodes:
            if i == j: 
                continue
            m.addConstr(s[i, j] + h[i] <= 1.0)
            m.addConstr(s[i, j] + h[j] <= 1.0)

    # flow conservation and capacity on hub links
    Wtot = float(W.sum())
    for i in nodes:
        for k in nodes:
            outflow = gp.quicksum(f[i, k, l] for l in nodes if l != k)
            inflow  = gp.quicksum(f[i, l, k] for l in nodes if l != k)
            used_out = gp.quicksum(W[i, j] * y[i, j, k, l] for j in nodes for l in nodes if (i, j, k, l) in y)
            used_in  = gp.quicksum(W[i, j] * y[i, j, l, k] for j in nodes for l in nodes if (i, j, l, k) in y)
            m.addConstr(inflow + used_out == outflow + used_in)

    for i in nodes:
        for k in nodes:
            for l in nodes:
                if k == l: 
                    continue
                m.addConstr(f[i, k, l] <= Wtot * z[k, l])

    # params
    m.Params.TimeLimit = timelimit
    m.Params.MIPGap = mipgap
    if threads is not None:
        m.Params.Threads = threads

    # Warm start
    if warm:
        Hset, Zset, Yset, Sset = warm_start_direct(R, F, G, q, alpha, Cmat, W)
        for k in nodes:
            h[k].start = 1.0 if k in Hset else 0.0
        for (k, l), var in z.items():
            if k == l: 
                var.start = 0.0
            else:
                var.start = 1.0 if (k, l) in Zset else 0.0
        for key, var in y.items():
            i, j, k, l = key
            var.start = 1.0 if (i, j, k, l) in Yset else 0.0
        for i in nodes:
            for j in nodes:
                if i == j: 
                    continue
                s[i, j].start = 1.0 if (i, j) in Sset else 0.0

    t0 = time()
    m.optimize()
    dt = time() - t0

    # results
    obj = None
    if m.SolCount > 0:
        obj = m.ObjVal

    served_hub = 0.0
    served_dir = 0.0
    open_dirs = []
    if m.SolCount > 0:
        for i in nodes:
            for j in nodes:
                if i == j: 
                    continue
                if s[i, j].X > 0.5:
                    served_dir += W[i, j]
                    open_dirs.append((i, j))
        for (i, j, k, l), var in y.items():
            if var.X > 0.5:
                served_hub += W[i, j]

    totalW = float(W.sum())
    sat_pct = 100.0 * (served_hub + served_dir) / totalW
    sat_dir_pct = 100.0 * served_dir / totalW

    # objective parts
    rev_total = (rev_hub.getValue() + rev_dir.getValue()) if m.SolCount > 0 else None
    return {
        "obj": obj,
        "sat_pct": sat_pct,
        "sat_dir_pct": sat_dir_pct,
        "n_dir": len(open_dirs),
        "time_sec": dt,
        "gap": m.MIPGap if m.SolCount > 0 else None,
        "rev": rev_total,
        "cost_cd": cost_cd.getValue() if m.SolCount > 0 else None,
        "cost_tr": cost_tr.getValue() if m.SolCount > 0 else None,
        "cost_dir": cost_dir.getValue() if m.SolCount > 0 else None,
        "fix_hub": fix_hub.getValue() if m.SolCount > 0 else None,
        "fix_link": fix_link.getValue() if m.SolCount > 0 else None,
        "fix_dir": fix_dir.getValue() if m.SolCount > 0 else None,
    }

# ----- parameter sweep -----
alphas = [0.2, 0.35, 0.5]
q_scales = [1.0, 0.5, 0.2]
gamma_fixed = 0.30  # keep same as best no-direct case

# Build scaled fixed costs for hubs/links
R_base, F_base, G_base, Q_base, _ = cab_params("high", "low", 0.2)
F_scaled = F_base * tau * gamma_fixed
G_scaled = G_base * tau * gamma_fixed

results = []
for a in alphas:
    for qs in q_scales:
        print(f"\n=== alpha={a}, q_scale={qs} ===")
        q_mat = construct_q_matrix(G_base, tau, qs)
        R_use = R_base  # revenues unchanged
        res = build_solve_multiple_direct(
            C_scaled, R_use, F_scaled, G_scaled, q_mat, a, W,
            timelimit=180, mipgap=1e-4, warm=True
        )
        res.update({"alpha": a, "q_scale": qs})
        print(res)
        results.append(res)

df_direct = pd.DataFrame(results)
df_direct



=== alpha=0.2, q_scale=1.0 ===
Set parameter TimeLimit to value 180
Set parameter MIPGap to value 0.0001
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (mac64[x86] - Darwin 22.6.0 22H527)

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Non-default parameters:
TimeLimit  180

Optimize a model with 64591 rows, 32622 columns and 202862 nonzeros
Model fingerprint: 0xe9d05412
Variable types: 15625 continuous, 16997 integer (16997 binary)
Coefficient statistics:
  Matrix range     [6e-05, 1e+00]
  Objective range  [9e-04, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

User MIP start did not produce a new incumbent solution
User MIP start violates constraint R674 by 1.000000000

Found heuristic solution: objective 45.7197842
Presolve removed 4499 rows and 1241 columns
Presolve time: 3.01s
Presolved: 60092 rows, 31381 columns, 194061 nonzeros
Variable types: 15000 continuous, 163

{'obj': 978.8705131752315, 'sat_pct': 93.363570300293, 'sat_dir_pct': 0.0, 'n_dir': 0, 'time_sec': 180.12292575836182, 'gap': 0.002286684452531826, 'rev': 1867.27140600586, 'cost_cd': 153.48099401267086, 'cost_tr': 376.7497679075139, 'cost_dir': 0.0, 'fix_hub': 156.6994322733201, 'fix_link': 201.47069863712446, 'fix_dir': 0.0, 'alpha': 0.2, 'q_scale': 1.0}

=== alpha=0.2, q_scale=0.5 ===
Set parameter TimeLimit to value 180
Set parameter MIPGap to value 0.0001
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (mac64[x86] - Darwin 22.6.0 22H527)

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Non-default parameters:
TimeLimit  180

Optimize a model with 64591 rows, 32622 columns and 202862 nonzeros
Model fingerprint: 0x3eae8502
Variable types: 15625 continuous, 16997 integer (16997 binary)
Coefficient statistics:
  Matrix range     [6e-05, 1e+00]
  Objective range  [9e-04, 2e+04]
  Bounds range     [1e+00

Presolve removed 3718 rows and 1137 columns
Presolve time: 2.57s
Presolved: 60873 rows, 31485 columns, 195102 nonzeros
Variable types: 15000 continuous, 16485 integer (16485 binary)

Deterministic concurrent LP optimizer: primal and dual simplex (primal and dual model)
Showing primal log only...

Root relaxation presolved: 60873 rows, 31485 columns, 195102 nonzeros

Concurrent spin time: 0.23s

Solved with dual simplex

Root relaxation: objective 9.890956e+02, 5729 iterations, 2.36 seconds (1.37 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  989.09563    0 1017  187.50059  989.09563   428%     -    5s
H    0     0                     797.7714474  989.09563  24.0%     -    5s
H    0     0                     826.2080569  989.09563  19.7%     -    5s
H    0     0                     828.1286537  989.09563  19.4%     -    5s
H    0     0                     95


Found heuristic solution: objective 45.7197842
Presolve removed 4499 rows and 1241 columns
Presolve time: 2.70s
Presolved: 60092 rows, 31381 columns, 194061 nonzeros
Variable types: 15000 continuous, 16381 integer (16381 binary)

Deterministic concurrent LP optimizer: primal and dual simplex (primal and dual model)
Showing primal log only...

Root relaxation presolved: 60092 rows, 31381 columns, 194061 nonzeros


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   10528    6.2779656e+02   0.000000e+00   1.583885e+06      5s
Concurrent spin time: 0.15s

Solved with primal simplex

Root relaxation: objective 7.598242e+02, 12109 iterations, 3.19 seconds (1.97 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  759.82415    0 1671   45.71978  759.82415  1562%     -    6s
H    0     0                     392.6225898  759.82415  93

User MIP start violates constraint R674 by 1.000000000

Found heuristic solution: objective 107.5856307
Presolve removed 4074 rows and 1191 columns
Presolve time: 3.06s
Presolved: 60517 rows, 31431 columns, 194648 nonzeros
Variable types: 15000 continuous, 16431 integer (16431 binary)

Deterministic concurrent LP optimizer: primal and dual simplex (primal and dual model)
Showing primal log only...

Root relaxation presolved: 60517 rows, 31431 columns, 194648 nonzeros


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
    8822    5.4920795e+02   0.000000e+00   2.354191e+06      5s
Concurrent spin time: 0.23s

Solved with primal simplex

Root relaxation: objective 7.616703e+02, 10944 iterations, 2.84 seconds (1.70 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  761.67026    0 1601  107.58563  761.67026   608%     -    6s
H  

...


    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  585.27972    0 1586   45.71978  585.27972  1180%     -    7s
H    0     0                     329.6035845  585.27972  77.6%     -    8s
H    0     0                     370.6229907  585.27972  57.9%     -    8s
H    0     0                     371.1797197  585.27972  57.7%     -    8s
H    0     0                     393.7625344  585.27972  48.6%     -    8s
     0     0  557.22519    0  943  393.76253  557.22519  41.5%     -   14s
     0     0  556.05072    0 1237  393.76253  556.05072  41.2%     -   17s
H    0     0                     430.0721047  556.03572  29.3%     -   18s
H    0     0                     435.6940357  556.03572  27.6%     -   18s
H    0     0                     449.0668073  556.03572  23.8%     -   18s
H    0     0                     450.7741499  556.03572  23.4%     -   18s
H    0     0           

Set parameter TimeLimit to value 180
Set parameter MIPGap to value 0.0001
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (mac64[x86] - Darwin 22.6.0 22H527)

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Non-default parameters:
TimeLimit  180

Optimize a model with 64591 rows, 32622 columns and 202862 nonzeros
Model fingerprint: 0xe9b6fa9d
Variable types: 15625 continuous, 16997 integer (16997 binary)
Coefficient statistics:
  Matrix range     [6e-05, 1e+00]
  Objective range  [9e-04, 5e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

User MIP start did not produce a new incumbent solution
User MIP start violates constraint R674 by 1.000000000

Found heuristic solution: objective 107.5856307
Presolve removed 4074 rows and 1191 columns
Presolve time: 2.74s
Presolved: 60517 rows, 31431 columns, 194648 nonzeros
Variable types: 15000 continuous, 16431 integer (16431 binary)

Dete

Best objective 5.504471743123e+02, best bound 5.504472996707e+02, gap 0.0000%
{'obj': 550.447174312274, 'sat_pct': 82.4737034967829, 'sat_dir_pct': 0.0, 'n_dir': 0, 'time_sec': 93.11208820343018, 'gap': 2.277393181845628e-07, 'rev': 1649.4740699356573, 'cost_cd': 237.71045566313336, 'cost_tr': 603.1354705956385, 'cost_dir': 0.0, 'fix_hub': 134.31379909141722, 'fix_link': 123.8671702731959, 'fix_dir': 0.0, 'alpha': 0.5, 'q_scale': 0.5}

=== alpha=0.5, q_scale=0.2 ===
Set parameter TimeLimit to value 180
Set parameter MIPGap to value 0.0001
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (mac64[x86] - Darwin 22.6.0 22H527)

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Non-default parameters:
TimeLimit  180

Optimize a model with 64591 rows, 32622 columns and 202862 nonzeros
Model fingerprint: 0x17fec2dd
Variable types: 15625 continuous, 16997 integer (16997 binary)
Coefficient statistics:
  Matrix rang

{'obj': 550.4470473599577, 'sat_pct': 82.4737034967829, 'sat_dir_pct': 0.0, 'n_dir': 0, 'time_sec': 98.09621405601501, 'gap': 4.583742994696528e-07, 'rev': 1649.4740699356573, 'cost_cd': 237.7103571329789, 'cost_tr': 603.1356960781094, 'cost_dir': 0.0, 'fix_hub': 134.31379909141722, 'fix_link': 123.8671702731959, 'fix_dir': 0.0, 'alpha': 0.5, 'q_scale': 0.2}


,obj,sat_pct,sat_dir_pct,n_dir,time_sec,gap,rev,cost_cd,cost_tr,cost_dir,fix_hub,fix_link,fix_dir,alpha,q_scale
0,978.870513,93.363570,0.0,0,180.122926,2.286684e-03,1867.271406,153.480994,376.749768,0.0,156.699432,201.470699,0.0,0.20,1.0
1,978.870526,93.363570,0.0,0,180.128607,1.588056e-03,1867.271406,153.480842,376.749907,0.0,156.699432,201.470699,0.0,0.20,0.5
2,978.870516,93.363570,0.0,0,180.139402,4.518802e-04,1867.271406,153.480962,376.749797,0.0,156.699432,201.470699,0.0,0.20,0.2
3,746.538805,85.929369,0.0,0,57.756407,1.190927e-07,1718.587380,191.446860,482.872794,0.0,141.775677,155.953245,0.0,0.35,1.0
4,746.538895,85.929369,0.0,0,60.196175,5.547139e-05,1718.587380,191.447006,482.872557,0.0,141.775677,155.953245,0.0,0.35,0.5
5,746.538768,85.929369,0.0,0,61.042371,1.686159e-07,1718.587380,191.446799,482.872892,0.0,141.775677,155.953245,0.0,0.35,0.2
6,550.447266,82.473703,0.0,0,131.163989,6.138882e-08,1649.474070,237.710527,603.135308,0.0,134.313799,123.867170,0.0,0.50,1.0
7,550.447174,82.473703,0.0,0,93.112088,2.277393e-07,1649.474070,237.710456,603.135471,0.0,134.313799,123.867170,0.0,0.50,0.5
8,550.447047,82.473703,0.0,0,98.096214,4.583743e-07,1649.474070,237.710357,603.135696,0.0,134.313799,123.867170,0.0,0.50,0.2
